ksekinisame apo to baseline
meta skeftika na valo ta idia pou eixe gia abstracts kai gia tous authors
tora 8a prospa8iso na kano hyperparameter tuning kai na xrisimopoiiso ki alla modela
Meta 8a prospa8iso na xrisimopoiiso cosine similrity me tin xrisi to word2vec
to cosine similarity den doulevei giati ta vector exoun diaforetika sized kati prepei na allakso

den exo kanei kapos to tfidf kai to cosine similarity tou na doulevei den katalavaino giati alla 8a epistrepso sintoma

afino stin akri gia ligo ta abstracts kai epikentronomai sto graph -> 8a kano node embedding meta edge embedding
telos prepei kapois na xrisimopoiisoume ta features pou dinei to networkx alla einai gtp gt dinei iterators opote prepei na vroume ena scriptaki pou 8a to sozei kapos



In [1]:
import networkx as nx
import gensim
import csv
import numpy as np
from random import randint
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import nltk
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity as cosim

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Create a graph
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)


Number of nodes: 138499
Number of edges: 1091955


In [4]:
n2v = Word2Vec.load('n2v_model.model')

In [5]:
# Read the abstract of each paper
abstracts = dict()
with open('abstracts.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [6]:
# Read the authors of each paper
authors = dict()
with open('authors.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, author = line.split('|--|')
        authors[int(node)] = author

In [7]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"'d", "'ll", "'re", "'s", "'ve", '``', 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would','”','“'])

In [8]:
lemmatizer = WordNetLemmatizer()

In [10]:
import re
re.findall(r"[a-z]\w+|[A-Z]\w+(?: [A-Z]\w+)?",authors[1])

['Jian', 'Xun Peng', 'Kang Li', 'De', 'Shuang Huang']

In [11]:
authors[1]

'Jian-Xun Peng,Kang Li,De-Shuang Huang\n'

In [12]:
word_tokenize(authors[1])

['Jian-Xun', 'Peng', ',', 'Kang', 'Li', ',', 'De-Shuang', 'Huang']

In [ ]:
text = NER

In [8]:
cluster = nx.clustering(G)

In [10]:

tokenized_abstracts = dict()
for node in abstracts:
    tokenized_abstracts[node] = []
    for sent in sent_tokenize(abstracts[node]):
        for i in word_tokenize(sent):
            if i.lower() in stop_words:
                continue
            else:
                tokenized_abstracts[node].append(i.lower())

In [12]:
for node in abstracts:
    print(authors[node] in abstracts[node])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [9]:
tokenized_authors = dict()
for node in authors:
    for word in word_tokenize(authors[node]):
        if word not in stop_words:
            if node not in tokenized_authors:
                tokenized_authors[node] = [word]
            else:
                tokenized_authors[node].append(word)

In [11]:
for node in abstracts:
    abstracts[node] = set(abstracts[node].split(' '))

In [12]:
for node in authors:
    authors[node] = set(authors[node].split(','))

Calculating the cosine similarity between the similar words of the abstracts
The vectors represent the frequency of the words in each abstracts

In [13]:
import math
epsilon = 1e-6
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / ((magA * magB) + epsilon)

In [14]:
rank = nx.pagerank(G)

In [15]:
h,a = nx.hits(G)

In [16]:
triangles = nx.triangles(G)

In [17]:
model = Word2Vec(tokenized_abstracts.values(), window=20, min_count=1, workers=-1,sg=1)

In [18]:
model_authors = Word2Vec(tokenized_authors.values(), window=5, min_count=1, workers=-1,sg=1)

In [19]:
#import taggeddocument
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [20]:
docs = [TaggedDocument(words=tokenized_abstracts[node], tags=[node]) for node in abstracts]
ath_doca = [TaggedDocument(words=tokenized_authors[node], tags=[node]) for node in authors]
d2v = Doc2Vec(docs, vector_size=100, window=10, min_count=1, workers= -1)
d2v_ath = Doc2Vec(ath_doca, vector_size=100, window=5, min_count=1, workers= -1)

In [21]:
from scipy.spatial.distance import cosine

In [22]:
for node in tokenized_abstracts:
    if tokenized_abstracts[node] == []:
        tokenized_abstracts[node] = ['none']

In [23]:
berts = np.load('embeddings.npy')

In [7]:
from sentence_transformers import SentenceTransformer, util
roberta = SentenceTransformer('stsb-roberta-large')

In [48]:
t5 = SentenceTransformer('sentence-t5-large')

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 83.9kB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 38.7kB/s]
Downloading: 100%|██████████| 3.15M/3.15M [00:02<00:00, 1.12MB/s]
Downloading: 100%|██████████| 2.02k/2.02k [00:00<00:00, 1.96MB/s]
Downloading: 100%|██████████| 1.39k/1.39k [00:00<00:00, 694kB/s]
Downloading: 100%|██████████| 461/461 [00:00<00:00, 462kB/s]
Downloading: 100%|██████████| 670M/670M [09:20<00:00, 1.20MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 52.9kB/s]
Downloading: 100%|██████████| 1.79k/1.79k [00:00<00:00, 1.78MB/s]
Downloading: 100%|██████████| 792k/792k [00:00<00:00, 1.07MB/s]
Downloading: 100%|██████████| 1.39M/1.39M [00:01<00:00, 921kB/s] 
Downloading: 100%|██████████| 1.92k/1.92k [00:00<00:00, 1.88MB/s]


In [8]:
string1 =abstracts[1]
string2 =abstracts[2]
string1

'This paper proposes a novel hybrid forward algorithm (HFA) for the construction of radial basis function (RBF) neural networks with tunable nodes. The main objective is to efficiently and effectively produce a parsimonious RBF neural network that generalizes well. In this study, it is achieved through simultaneous network structure determination and parameter optimization on the continuous parameter space. This is a mixed integer hard problem and the proposed HFA tackles this problem using an integrated analytic framework, leading to significantly improved network performance and reduced memory usage for the network construction. The computational complexity analysis confirms the efficiency of the proposed algorithm, and the simulation results demonstrate its effectiveness\n'

In [9]:
string2

'Modern CCD cameras are usually capable of a spatial accuracy greater than 1/50 of the pixel size. However, such accuracy is not easily attained due to various error sources that can affect the image formation process. Current calibration methods typically assume that the observations are unbiased, the only error is the zero-mean independent and identically distributed random noise in the observed image coordinates, and the camera model completely explains the mapping between the 3D coordinates and the image coordinates. In general, these conditions are not met, causing the calibration results to be less accurate than expected. In the paper, a calibration procedure for precise 3D computer vision applications is described. It introduces bias correction for circular control points and a nonrecursive method for reversing the distortion model. The accuracy analysis is presented and the error sources that can reduce the theoretical accuracy are discussed. The tests with synthetic images ind

In [49]:
embedding1 = t5.encode(string1, convert_to_tensor=True)
embedding2 = t5.encode(string2, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Similarity score:", cosine_scores.item())

Similarity score: 0.8261343836784363


In [10]:
embedding1 = roberta.encode(string1, convert_to_tensor=True)
embedding2 = roberta.encode(string2, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Similarity score:", cosine_scores.item())

Similarity score: 0.5287594795227051


In [46]:
model.wv.n_similarity(tokenized_abstracts[1], tokenized_abstracts[2])

0.108180806

In [47]:
util.pytorch_cos_sim(berts[1], berts[2]).item()

0.8010352849960327

In [42]:
cosine(berts[1], berts[2])

0.19896531105041504

In [27]:
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) sum of number of unique terms of the two nodes' abstracts
# (2) absolute value of difference of number of unique terms of the two nodes' abstracts
# (3) number of common terms between the abstracts of the two nodes
# (4) sum of number of unique terms of the two nodes' authors
# (5) absolute value of difference of number of unique terms of the two nodes' authors
# (6) 

X_train = np.zeros((2*m, 28))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
for i,edge in tqdm(enumerate(G.edges()), desc='Training', total=m):
    # an edge
    X_train[i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X_train[i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    X_train[i,2] = len(abstracts[edge[0]].intersection(abstracts[edge[1]]))
    X_train[i,3] = len(authors[edge[0]]) + len(authors[edge[1]])
    X_train[i,4] = abs(len(authors[edge[0]]) - len(authors[edge[1]]))
    X_train[i,5] = len(authors[edge[0]].intersection(authors[edge[1]]))
    X_train[i,6] = counter_cosine_similarity(Counter(tokenized_authors[edge[0]]), Counter(tokenized_authors[edge[1]]))
    X_train[i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[edge[0]]), Counter(tokenized_abstracts[edge[1]]))
    X_train[i,8] = rank[edge[0]] + rank[edge[1]]
    X_train[i,9] = abs(rank[edge[0]] - rank[edge[1]])
    X_train[i,10] = cluster[edge[0]] + cluster[edge[1]]
    X_train[i,11] = abs(cluster[edge[0]] - cluster[edge[1]])
    X_train[i,12] = h[edge[0]] + h[edge[1]]
    X_train[i,13] = abs(h[edge[0]] - h[edge[1]])
    X_train[i,14] = triangles[edge[0]] + triangles[edge[1]]
    X_train[i,15] = abs(triangles[edge[0]] - triangles[edge[1]])
    X_train[i,16] = nx.degree(G, edge[0]) + nx.degree(G, edge[1])
    X_train[i,17] = abs(nx.degree(G, edge[0]) - nx.degree(G, edge[1]))
    X_train[i,18] = len(list(nx.common_neighbors(G, edge[0], edge[1])))
    X_train[i,19] = n2v.wv.n_similarity(G[edge[0]], G[edge[1]])
    X_train[i,20] = n2v.wv.similarity(edge[0], edge[1])
    X_train[i,21] = model.wv.n_similarity(tokenized_abstracts[edge[0]], tokenized_abstracts[edge[1]])
    X_train[i,22] = model_authors.wv.n_similarity(tokenized_authors[edge[0]], tokenized_authors[edge[1]])
    X_train[i,23] = a[edge[0]] + a[edge[1]]
    X_train[i,24] = abs(a[edge[0]] - a[edge[1]])
    X_train[i,25] = cosine(d2v[edge[0]], d2v[edge[1]])
    X_train[i,26] = cosine(d2v_ath[edge[0]], d2v_ath[edge[1]])
    X_train[i,27] = cosine(berts[edge[0]], berts[edge[1]])
    y_train[i] = 1

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    while G.has_edge(n1, n2):
        n1 = randint(0, n-1)
        n2 = randint(0, n-1)
    X_train[m+i,0] = len(abstracts[n1]) + len(abstracts[n2])
    X_train[m+i,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
    X_train[m+i,2] = len(abstracts[n1].intersection(abstracts[n2]))
    X_train[m+i,3] = len(authors[n1]) + len(authors[n2])
    X_train[m+i,4] = abs(len(authors[n1]) - len(authors[n2]))
    X_train[m+i,5] = len(authors[n1].intersection(authors[n2]))
    X_train[m+i,6] = counter_cosine_similarity(Counter(tokenized_authors[n1]), Counter(tokenized_authors[n2]))
    X_train[m+i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[n1]), Counter(tokenized_abstracts[n2]))
    X_train[m+i,8] = rank[n1] + rank[n2] # sum of ranks of the two nodes
    X_train[m+i,9] = abs(rank[n1] - rank[n2]) # absolute value of difference of ranks of the two nodes
    X_train[m+i,10] = cluster[n1] + cluster[n2] # sum of clusters of the two nodes
    X_train[m+i,11] = abs(cluster[n1] - cluster[n2]) # absolute value of difference of clusters of the two nodes
    X_train[m+i,12] = h[n1] + h[n2] # sum of hubs of the two nodes
    X_train[m+i,13] = abs(h[n1] - h[n2]) # absolute value of difference of hubs of the two nodes
    X_train[m+i,14] = triangles[n1] + triangles[n2] # sum of triangles of the two nodes
    X_train[m+i,15] = abs(triangles[n1] - triangles[n2]) # absolute value of difference of triangles of the two nodes
    X_train[m+i,16] = nx.degree(G, n1) + nx.degree(G, n2) # sum of degrees of the two nodes
    X_train[m+i,17] = abs(nx.degree(G, n1) - nx.degree(G, n2)) # absolute value of difference of degrees of the two nodes
    X_train[m+i,18] = len(list(nx.common_neighbors(G, n1, n2))) # number of common neighbors of the two nodes
    X_train[m+i,19] = n2v.wv.n_similarity(G[n1], G[n2]) # cosine similarity between the two nodes
    X_train[m+i,20] = n2v.wv.similarity(n1, n2) # cosine similarity between the two nodes
    X_train[m+i,21] = model.wv.n_similarity(tokenized_abstracts[n1], tokenized_abstracts[n2]) # cosine similarity between the two nodes
    X_train[m+i,22] = model_authors.wv.n_similarity(tokenized_authors[n1], tokenized_authors[n2]) # cosine similarity between the two nodes
    X_train[m+i,23] = a[n1] + a[n2] # sum of authors of the two nodes
    X_train[m+i,24] = abs(a[n1] - a[n2]) # absolute value of difference of authors of the two nodes
    X_train[m+i,25] = cosine(d2v[n1], d2v[n2]) # cosine similarity between the two nodes
    X_train[m+i,26] = cosine(d2v_ath[n1], d2v_ath[n2]) # cosine similarity between the two nodes
    X_train[m+i,27] = cosine(berts[n1], berts[n2]) # cosine similarity between the two nodes
    y_train[m+i] = 0

Training: 100%|██████████| 1091955/1091955 [44:43<00:00, 406.96it/s] 


In [28]:
print('Size of training matrix:', X_train.shape)

Size of training matrix: (2183910, 28)


In [29]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

In [30]:
# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 28))
for i,node_pair in tqdm(enumerate(node_pairs), desc='Creating test matrix', total=len(node_pairs)):
    X_test[i,0] = len(abstracts[node_pair[0]]) + len(abstracts[node_pair[1]])
    X_test[i,1] = abs(len(abstracts[node_pair[0]]) - len(abstracts[node_pair[1]]))
    X_test[i,2] = len(abstracts[node_pair[0]].intersection(abstracts[node_pair[1]]))
    X_test[i,3] = len(authors[node_pair[0]]) + len(authors[node_pair[1]])
    X_test[i,4] = abs(len(authors[node_pair[0]]) - len(authors[node_pair[1]]))
    X_test[i,5] = len(authors[node_pair[0]].intersection(authors[node_pair[1]]))
    X_test[i,6] = counter_cosine_similarity(Counter(tokenized_authors[node_pair[0]]), Counter(tokenized_authors[node_pair[1]]))
    X_test[i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[node_pair[0]]), Counter(tokenized_abstracts[node_pair[1]]))
    X_test[i,8] = rank[node_pair[0]] + rank[node_pair[1]]
    X_test[i,9] = abs(rank[node_pair[0]] - rank[node_pair[1]])
    X_test[i,10] = cluster[node_pair[0]] + cluster[node_pair[1]]
    X_test[i,11] = abs(cluster[node_pair[0]] - cluster[node_pair[1]])
    X_test[i,12] = h[node_pair[0]] + h[node_pair[1]]
    X_test[i,13] = abs(h[node_pair[0]] - h[node_pair[1]])
    X_test[i,14] = triangles[node_pair[0]] + triangles[node_pair[1]]
    X_test[i,15] = abs(triangles[node_pair[0]] - triangles[node_pair[1]])
    X_test[i,16] = nx.degree(G, node_pair[0]) + nx.degree(G, node_pair[1])
    X_test[i,17] = abs(nx.degree(G, node_pair[0]) - nx.degree(G, node_pair[1]))
    X_test[i,18] = len(list(nx.common_neighbors(G, node_pair[0], node_pair[1])))
    X_test[i,19] = n2v.wv.n_similarity(G[node_pair[0]], G[node_pair[1]])
    X_test[i,20] = n2v.wv.similarity(node_pair[0], node_pair[1])
    X_test[i,21] = model.wv.n_similarity(tokenized_abstracts[node_pair[0]], tokenized_abstracts[node_pair[1]])
    X_test[i,22] = model_authors.wv.n_similarity(tokenized_authors[node_pair[0]], tokenized_authors[node_pair[1]])
    X_test[i,23] = a[node_pair[0]] + a[node_pair[1]]
    X_test[i,24] = abs(a[node_pair[0]] - a[node_pair[1]])
    X_test[i,25] = cosine(d2v[node_pair[0]], d2v[node_pair[1]])
    X_test[i,26] = cosine(d2v_ath[node_pair[0]], d2v_ath[node_pair[1]])
    X_test[i,27] = cosine(berts[node_pair[0]], berts[node_pair[1]])
print('Size of test matrix:', X_test.shape)

Creating test matrix: 100%|██████████| 106692/106692 [02:10<00:00, 819.65it/s]

Size of test matrix: (106692, 28)


In [31]:
X_train, y_train = shuffle(X_train, y_train)

In [38]:
import lightgbm as lgb

lb = lgb.LGBMClassifier(objective='binary',boosting_type='dart', n_estimators=1000, max_bin = 300 ,n_jobs=-1, device='cpu')
lb.fit(X_train, y_train)

y_pred_lgb = lb.predict_proba(X_test)
y_pred_lgb = y_pred_lgb[:,1]

In [39]:
# Write predictions to a file
predictions = zip(range(len(y_pred_lgb)), y_pred_lgb)
with open("submissions_6.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)